# RAG(Retrieval-Augmented Generation)

In [1]:
import ollama
import chromadb

In [2]:
# 文書のリスト
documents = [
	"烏: 常に正しい答えを返す",
	"鶏: 正確でない答えを返す",
]

print(documents)

['烏: 常に正しい答えを返す', '鶏: 正確でない答えを返す']


In [3]:
# Chromadbクライアントを作成
client = chromadb.Client()

# "docs"という名前のコレクションを作成
collection = client.get_or_create_collection(name="docs")

# 各文書をベクトル化したのちにChromaDBに保存
for i, d in enumerate(documents):
    # 文書のベクトル表現を生成
    response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
    embedding = response["embedding"]
    
    # 文書をChromaDBに追加
    collection.upsert(
        ids=["id."+str(i+1)],
        embeddings=[embedding],
        documents=[d]
    )

print(collection.peek()["ids"])
print(collection.peek()["documents"])

['id.1', 'id.2']
['烏: 常に正しい答えを返す', '鶏: 正確でない答えを返す']


In [4]:
# プロンプトへの入力
prompt = "鶏は信用できるか？"

# プロンプトのベクトル表現を生成し、最も類似した文書を取得
response = ollama.embeddings(
    prompt=prompt,
    model="mxbai-embed-large"
)
results = collection.query(
    query_embeddings=[response["embedding"]],
    n_results=1
)

# 検索結果の取得
data = results['documents'][0][0]
print(data)

鶏: 正確でない答えを返す


In [5]:
# RAGの実行
output = ollama.generate(
    model="llama3",
    prompt=f"「{data}」という情報を元に以下の質問に日本語で回答してください： {prompt}"
)

# 生成された応答の出力
print(output['response'])

🐓😏 According to the information, 「鶏: 正確でない答えを返す」という（The chicken: returns an incorrect answer）。 So, based on this definition, I would say... 🤔

鶏は信用できない（The chicken is not trustworthy or reliable。🐓😜
